In [1703]:
import numpy as np
import pandas as pd
# import sklearn.datasets as data

In [1704]:
df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')
df.head()
print(df.shape)


(305434, 45)


Dropped Meldingstidspunkt, Starttidspunkt, Art - FDIR and strings from dataset due to how the data scaling generaly dislikes strings!
After dropping stings the scores went from -11% to 20%! (Both test and training) 

Maybe use one-hot encoding to include it anyway?

In [1705]:
#Sile ut informasjon:
df = df[(df['Art - FDIR (kode)'] == 1022) | (df['Art - FDIR (kode)'] == 1027)] # Fiske som fanger hyse eller torsk. "751 - lodde, 1022 Torsk"
area = [7, 8, 28, 30, 40]
df = df[(df['Redskap FDIR (kode)'] == 51) & (df['Hovedområde start (kode)'].isin(area))] # Fiske med bunntrål - 51, snurrevad - 11
df['Bruttotonnasje Kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen']) # Slår sammen kolonnene.
# Dropper unødvendig informasjon. Tabellen inneholder masse duplikater. som egen kolonne for navn og for kode
# Forskjellige klassifiseringsmåter. Holder på FDIR sine.
df.drop(columns = ['Meldingsdato', 
                   'Meldingsklokkeslett', 
                   'Startdato',  
                   'Hovedområde start', 
                   'Lokasjon start (kode)', 
                   'Stoppdato', 
                   'Stoppklokkeslett', 
                   'Fangstår', 
                   'Hovedområde stopp (kode)', 
                   'Hovedområde stopp', 
                   'Lokasjon stopp (kode)', 
                   'Redskap FAO (kode)', 
                   'Redskap FAO', 
                   'Redskap FDIR', 
                   'Hovedart FAO (kode)', 
                   'Hovedart FAO', 
                   'Art FAO (kode)',
                   'Art FAO',
                   'Art - gruppe', 
                   'Lengdegruppe', 
                   'Bredde', 
                   'Fartøylengde',
                   'Bruttotonnasje 1969',
                   'Bruttotonnasje annen',

                    #Columns removed for training of MLPRegressor neural network
                   'Meldingstidspunkt',
                   'Starttidspunkt',
                   'Startklokkeslett',
                   'Stopptidspunkt',
                   'Art - FDIR',
                   'Melding ID',
                   'Havdybde start',# Filter and remove incorrect values
                   'Havdybde stopp',# Filter and remove incorrect values
                   'Redskap FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                  #  'Bruttotonnasje Kombinert',
                    'Hovedart - FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                    'Art - FDIR (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
                    'Art - gruppe (kode)', #Fjernet grunnet gjentagene data som ikke gav noe verdi
    ], inplace=True)

# Gruppere bruttotonnasje:
bins = [x for x in range(0, 4800, 100)] # Definerer gruppene
labels = range(1, len(bins))  # nummererer gruppene.
# Categorize 'Bruttotonnasje Kombinert' into the defined bins
df['Bruttotonnasje Gruppert'] = pd.cut(df['Bruttotonnasje Kombinert'], bins=bins, labels=labels)
# Teller grupperingene
df['Bruttotonnasje Gruppert'].value_counts()

Bruttotonnasje Gruppert
6     992
3     950
2     618
21    465
4     433
35    312
14    305
32    261
5     257
25    233
1     210
12    191
16    186
15    181
40    164
38     78
26     50
7      49
23     45
19     44
45     17
17      8
42      5
44      0
46      0
41      0
33      0
37      0
36      0
43      0
34      0
39      0
24      0
31      0
30      0
29      0
28      0
27      0
22      0
20      0
18      0
13      0
11      0
10      0
9       0
8       0
47      0
Name: count, dtype: int64

In [1706]:
df.head()

,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),Varighet,Stopposisjon bredde,Stopposisjon lengde,Trekkavstand,Rundvekt,Lengdegruppe (kode),Bruttotonnasje Kombinert,Bruttotonnasje Gruppert
353,57.588,6.834,8.0,692,57.534,7.466,38303.0,114.0,4.0,267.0,3
635,61.221,2.181,28.0,330,61.481,1.753,36931.0,100.0,5.0,2053.0,21
1000,58.318,6.210,8.0,71,58.328,6.200,1259.0,10.0,3.0,79.0,1
1001,58.318,6.210,8.0,71,58.328,6.200,1259.0,8.0,3.0,79.0,1
1149,63.411,5.656,7.0,302,63.346,5.699,7558.0,72.0,5.0,536.0,6


In [1707]:
print(df.shape)
df.head()
y = df['Rundvekt'] 
X = df.drop('Rundvekt', axis=1) #Dropping rundvekt since that's what i'm training the model on



(6054, 11)


In [1708]:
# X.shape = (6054, 23)
# y.shape = (6054,)

In [1709]:
from sklearn.datasets import make_regression
# X = df.drop('Rundvekt', axis=1)
# y = df['Rundvekt']
X, y = make_regression(n_samples=200, random_state=1) #Might make it overfit!
# print(X,y)

In [1710]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 1235)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [1711]:
#MinMax Scaler works best with MLPRegressor

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# scaler.fit(X)
# X = scaler.transform(X)
X_train_minmax = scaler.fit_transform(X_train)
X_test_minmax = scaler.fit_transform(X_test)

# print(X)

In [1712]:
#Turns out standard scaler does horrible tings to MLPregressor! it does best with values between 0 and 1

# from sklearn.preprocessing import StandardScaler
# sc=StandardScaler()

# scaler = sc.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [1713]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes = [20,10], #First hidden layer has 20 neurons, second hidden layer has 10 neurons
                  activation = 'relu', alpha = 0.1, 
                  max_iter = 2000, #Max iterations you do. AKA max epocs
                #   early_stopping=True, #Can't use early stopping due to massive loss values in Rundvekt
                #   tol=
                  learning_rate_init=0.1,#Initial learning rate, adaptive adapts this learning rate to minimize loss
                  learning_rate="adaptive",
                  solver = 'adam', 
                  verbose = False)
#activation logistic tanh relu identity
#solver adam lbfgs sgd 

In [1714]:

#Test score without data leak. (using minmax after splitting)
# nn.fit(X_train, y_train)
nn.fit(X_train_minmax, y_train)
print("Neural network, MLPRegressor")
print("Test score",  nn.score(X_test_minmax, y_test))
print("Train  score", nn.score(X_train_minmax, y_train)) #Fix training score, might be wrong due to testing on training data


# print("Neural network, MLPRegressor")
# print("Test score",  nn.score(X_test, y_test))
# print("Train  score", nn.score(X_train, y_train)) #Fix training score, might be wrong due to testing on training data

# print("FIXED???? Neural network, MLPRegressor")
# print("Test score",  nn.score(X_test_scaled, y_test))
# print("Train  score", nn.score(X_train_scaled, y_train)) #Fix training score, might be wrong due to testing on training data


Neural network, MLPRegressor
Test score 0.707241424632056
Train  score 0.942746221707945
train minmax [0.64901324 0.70982269 0.54777129 0.2056272  0.13653067 0.55248075
 0.52689285 0.33729798 0.51080482 0.20913965 0.52155672 0.23643919
 0.45259559 0.73214865 0.60134079 0.35528118 0.49752182 0.54760864
 0.65095313 0.48025504 0.59377543 0.42403629 0.7328132  0.93750971
 0.59654453 0.44195873 0.68081658 0.57298198 0.22143087 0.
 0.21793211 0.19208665 0.81682314 0.63981844 0.50328424 0.36491212
 0.47247336 0.07389462 0.31925198 0.69094653 0.4891328  0.4502615
 0.33714277 0.72871916 0.54705959 0.12901696 0.72408443 0.63531096
 0.44529567 0.27132054 0.60139374 0.54242235 0.2838365  0.85928494
 0.37633475 0.53022333 0.37325079 0.26774468 0.52893178 0.45792583
 0.32469858 0.39583691 0.30239706 0.61099672 0.39163714 0.17890702
 0.08823157 0.47185951 0.279634   0.16612655 0.84753618 0.52511866
 0.79577888 0.33409658 0.23696592 0.74323173 0.50376923 0.56704559
 0.3889209  0.64593144 0.43659257 0.

Oppsummering:
https://stackoverflow.com/questions/62131266/mlpregressor-working-but-results-dont-make-any-sense
It doesn't make any sense to predict on your training data. That provides no value, as it is testing the same data it learned from and should predict 100%. That is an example of 



important and relevant properties of the data
how you preprocessed data like which features you selected, did you do dimension reduction,
how you reformatted data, etc.
how you decided on parameters for your machine learning models,
if you used any regularization techniques? In case how.
how the methods were measured and compared


Turns out trying a max_iter of 1000 was a bit dumb! It wasnt enough iterations for the model to become optimized (converge)
To fix it i changed max_iter to 5000 and realized that max_iter isn't the amount of iterations you do! it's the max amount of iterations you do. I feel dumb
Iterations will be 5000 unless stated otherwise

With 5000 max iterations
Test score 0.15868380817426364
Train  score 0.2402524144823477

With 1000 max iterations
Test score 0.023895760424976542
Train  score 0.044879597360761636

Test 2
removed 'Redskap FDIR (kode)', 'Bruttotonnasje Kombinert',
Test score 0.22802640821841036
Train  score 0.17609209269226367
Eval: Turns out bruttotonnasje is important for amount of fish caught. I can't say i'm surprised

Test 3 max_iter = 10 000
Re-added 'Bruttotonnasje Kombinert'
Kept 'Redskap FDIR (kode)'
Turns out 5000 iterations wasn't enough! this will hopefully mean we have a smaller loss value and can thus get a higher accuracy as it converges better!
Changed to 10 000 iterations max
Test score 0.2183750907000509
Train  score 0.22780564641998635
The training score seems to have increased!
Running it again made Test score = 0.15405352930651073. Removing it seems to cause instability. i will keep it for now

Test 3
added a regression thingy and increased to 100 000 iterations (can probobaly lower it to 10 000 now)
Test score 0.3805415453308878
Train  score 0.9885294076382664
Obviously overtrained! but the test score is getting better!


Test 4
Fjernet:
Hovedart - FDIR (kode)
Art - FDIR (kode)
Art - gruppe (kode)
Dette er grunnet at disse verdiene ikke gav mye unik informasjon å lære fra. 

Test score 0.4551050579085447
Train  score 0.9926394850239878
Minimal økning, det var i alle fall ingen nedtur i nøyaktigheten. men den er enda overfitted

Max_iter = 1000
removed: Melding ID
Test score 0.038746802791864265
Train  score 0.04016411056751845


Max_iter = 1000
Test score 0.02857554746080171
Train  score 0.04388748121910424
Changing iter to 1000 with a good score is the goal!

V2, removed linear regression thingy
Max_iter = 5000
Test score 0.18474766635857465
Train  score 0.26563262873718185



Without removed collumns
Test score 0.03306077219837722
Train  score 0.07260871122128132

with removed collumnds (linear regression collumns)
Test score 0.034291607331412455
Train  score 0.07651381759634102